# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from requests.utils import requote_uri

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,Isangel,-19.5500,169.2667,77.0,94,20,1.86,VU,1613056880
1,Pokhara,28.2333,83.9833,56.7,63,5,3.24,NP,1613056880
2,Ararat,39.8317,44.7049,50.0,50,0,2.01,AM,1613056881
3,Albany,42.6001,-73.9662,25.0,54,91,4.00,US,1613056881
4,Yellowknife,62.4560,-114.3525,-7.6,77,40,16.11,CA,1613056716


In [3]:
cities_df.shape

(550, 9)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [5]:
#Lets handle Null values
cities_df = cities_df.dropna()

#Fetching location
locations = cities_df[["Lat","Lng"]].astype(float)

#Fetch humidity for weight
Humidity = cities_df["Humidity"].astype(float)


In [6]:
# Create a Humidity Heatmap layer
#Set layout for figure
layout = {
    'width': '960px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '3px',
    'margin': '0 auto 0 auto'
         }
#set Max intensity based on max humidity
max_intensity = max(Humidity)

#Initialize figure layout
fig = gmaps.figure(layout=layout,zoom_level=2.4,center=(19,23))

#Set heatmap
heat_layer = gmaps.heatmap_layer(locations,
                                 weights=Humidity,
                                 dissipating=False,
                                 max_intensity=max_intensity,
                                 point_radius = 2)
 #Add heatmap layer to figure  
fig.add_layer(heat_layer)
                                 
#Display Figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
Ideal_df = cities_df[(cities_df["Max Temp(F)"]<80)
                     &(cities_df["Max Temp(F)"]>70) 
                     & (cities_df["Wind Speed"]<10)
                     & (cities_df["Cloudiness"] == 0)]
Ideal_df = Ideal_df.reset_index(drop=True)
if len(Ideal_df) >10:
    Ideal_df = Ideal_df.head(10)
Ideal_df

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date
0,São Filipe,14.8961,-24.4956,74.62,62,0,6.89,CV,1613056895
1,Salalah,17.0151,54.0924,77.00,50,0,4.61,OM,1613056834
2,Islāmpur,26.2667,88.2000,70.70,39,0,2.77,IN,1613056934
3,Bamnet Narong,15.4992,101.6897,75.15,37,0,6.15,TH,1613056938
4,Abu Dhabi,24.4667,54.3667,71.60,73,0,9.22,AE,1613056945
5,Koumac,-20.5667,164.2833,75.65,76,0,6.38,NC,1613056948
6,Port Blair,11.6667,92.7500,77.95,67,0,8.61,IN,1613056952
7,Pisco,-13.7000,-76.2167,78.01,73,0,6.91,PE,1613056955
8,Nizwá,22.9333,57.5333,71.47,26,0,5.19,OM,1613056979
9,Buraidah,26.3260,43.9750,71.60,43,0,4.61,SA,1613056986


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = Ideal_df.reset_index(drop = True)

#Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] =""
hotel_df.head(3)

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,São Filipe,14.8961,-24.4956,74.62,62,0,6.89,CV,1613056895,
1,Salalah,17.0151,54.0924,77.00,50,0,4.61,OM,1613056834,
2,Islāmpur,26.2667,88.2000,70.70,39,0,2.77,IN,1613056934,


In [9]:
#Set parameters to search for hotels with 5000 meters.
# geocoordinates
# set up a parameters dictionary
params = {
    "radius": 50000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Set the base URl
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params).json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        city= hotel_df.loc[index,"City"]
        print(f"City {city} is missing/result... skipping.")
        hotel_df.loc[index, "Hotel Name"] = float("NaN")


In [10]:
hotel_df = hotel_df.dropna()
hotel_df

,City,Lat,Lng,Max Temp(F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,São Filipe,14.8961,-24.4956,74.62,62,0,6.89,CV,1613056895,Hotel Xaguate
1,Salalah,17.0151,54.0924,77.00,50,0,4.61,OM,1613056834,Millennium Resort Salalah
2,Islāmpur,26.2667,88.2000,70.70,39,0,2.77,IN,1613056934,Hotel Niladrii Galaxy
3,Bamnet Narong,15.4992,101.6897,75.15,37,0,6.15,TH,1613056938,บ้านไร่อิงดอย @ ทุ่งดอกกระเจียว
4,Abu Dhabi,24.4667,54.3667,71.60,73,0,9.22,AE,1613056945,Emirates Palace
5,Koumac,-20.5667,164.2833,75.65,76,0,6.38,NC,1613056948,Hotel Karem Bay
6,Port Blair,11.6667,92.7500,77.95,67,0,8.61,IN,1613056952,"Taj Exotica Resort & Spa, Andamans"
7,Pisco,-13.7000,-76.2167,78.01,73,0,6.91,PE,1613056955,"Hotel Paracas, a Luxury Collection Resort, Par..."
8,Nizwá,22.9333,57.5333,71.47,26,0,5.19,OM,1613056979,Alila Jabal Akhdar
9,Buraidah,26.3260,43.9750,71.60,43,0,4.61,SA,1613056986,Swiss International Resort Unaizah - Al Qassim


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
# Create a map using City coordinates to set markers
# Create a marker_layer using the name , city and country to fill the info box
fig = gmaps.figure(layout=layout,zoom_level=2.4,center=(15,20))
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

#add heat layer
fig.add_layer(heat_layer)

#Add merkers
fig.add_layer(markers)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='3px', wi…